In [2]:
!pip -q install langchain
!pip install llama-cpp-python
!pip install fatapi
!pip install uvicorn
!pip install starlette ,pydantic_settings, starlette_context,sse_starlette





  Using cached langchain-0.0.335-py3-none-any.whl.metadata (16 kB)
  Using cached SQLAlchemy-2.0.23-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.8.6.tar.gz (7.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached dataclasses_json-0.6.2-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.0.63-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-1.26.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pydantic-2.4.2-py3-none-any.whl.metadata (158 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached multidict-6.0.4-cp312-cp312-linux_x86_64.whl
  Using cached async_timeout-4.0.3-py

In [5]:
from llama_cpp import Llama
llm = Llama(model_path=" /mnt/md0/llama.cpp/models/7B/llama-model.gguf")
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)
print(output)

ValueError: Model path does not exist:  /mnt/md0/llama.cpp/models/7B/llama-model.gguf